### Create data dictionary for w2v embeddings

In [1]:
import pandas as pd
import urllib
import pyodbc
import pandas as pd
import random
import time
from random import randrange, uniform
from datetime import timedelta, datetime
import operator as op

dir_to_parse = '/Volumes/Data/AHC IE DATA REQUESTS/PakhomovS-U01/data_extract/'

In [2]:
# create connection string
# u01: Que1oow3aik0ia4Ak8hee^quo(ecae
# https://www.linkedin.com/pulse/connecting-sql-database-python-patrick-j-ryan/

params = 'DRIVER={ODBC Driver 17 for SQL Server};' \
         'SERVER=rsp-hr-db1.ahc.umn.edu;' \
         'PORT=1433;' \
         'DATABASE=pakhomovs-u01;' \
         'UID=U01;' \
         'PWD=uftGyHBkQ3u92LJB;'
         #'PWD=W2B8GWcAFWb_#8JrP;' 
         #'PWD=W2B_GWcAFWb_8JrP;' 
    
#conn = pyodbc.connect(params)  

params = urllib.parse.quote_plus(params)

#print(params)

from sqlalchemy import create_engine

engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params, fast_executemany=True)

print(engine)

Engine(mssql+pyodbc:///?odbc_connect=DRIVER={ODBC Driver 17 for SQL Server};SERVER=rsp-hr-db1.ahc.umn.edu;PORT=1433;DATABASE=pakhomovs-u01;UID=U01;PWD=uftGyHBkQ3u92LJB;)


In [3]:
#test data set group of patients 
codes = pd.read_csv(dir_to_parse + 'top-500-icd9-test.txt', engine='python', sep="|")
codes.head()
len(set(codes['service_id'].to_list()))

# write to db for use in pulling artifacts and service dates
# icd['service_id'].drop_duplicates().to_sql('initial_test_msi_service_id', engine)

15656

In [4]:
# read artifacts
df =  pd.read_csv(dir_to_parse + 'top-500-artifacts-test.txt',sep="|")
print(df.head())
print(df.columns)

   engine_id    note_id   patient_id   service_id      item  begin_span  \
0          2  315853867   3822879938  10148425300  C1961131        1250   
1          2  339277069   3823428289  10165647210  C0699129       12991   
2          2  393164995  19744085333  19814035865  C0262627        7411   
3          2  330657678   3823328041  10129396291  C0001554         509   
4          2  812461320   3823117966  49106721653  C0443288        3624   

   end_span                                         attributes  
0      1255       {'score': -861, 'preferred': 'Cough, CTCAE'}  
1     12999          {'score': -1000, 'preferred': 'Coumadin'}  
2      7417            {'score': -1000, 'preferred': 'Seroma'}  
3       519  {'score': -861, 'preferred': 'Administration o...  
4      3635         {'score': -827, 'preferred': 'Restricted'}  
Index(['engine_id', 'note_id', 'patient_id', 'service_id', 'item',
       'begin_span', 'end_span', 'attributes'],
      dtype='object')


In [5]:
# further filter set for testing by aadding slice
case_filter = list(set(codes['patient_id'].to_list()))#[0:2]

#case_filter

In [6]:
test = df[df['patient_id'].isin(case_filter)].copy()
test

,engine_id,note_id,patient_id,service_id,item,begin_span,end_span,attributes
0,2,315853867,3822879938,10148425300,C1961131,1250,1255,"{'score': -861, 'preferred': 'Cough, CTCAE'}"
1,2,339277069,3823428289,10165647210,C0699129,12991,12999,"{'score': -1000, 'preferred': 'Coumadin'}"
2,2,393164995,19744085333,19814035865,C0262627,7411,7417,"{'score': -1000, 'preferred': 'Seroma'}"
3,2,330657678,3823328041,10129396291,C0001554,509,519,"{'score': -861, 'preferred': 'Administration o..."
4,2,812461320,3823117966,49106721653,C0443288,3624,3635,"{'score': -827, 'preferred': 'Restricted'}"
...,...,...,...,...,...,...,...,...
15491998,2,371756778,3823318351,10121826985,C0043210,5613,5618,"{'score': -812, 'preferred': 'Woman'}"
15491999,2,534263402,3823024374,39442251018,C0002871,5656,5662,"{'score': -1000, 'preferred': 'Anemia'}"
15492000,2,472131061,9191984973,26008699914,C0521421,2624,2628,"{'score': -1000, 'preferred': 'Entire ear'}"
15492001,2,484598355,3824113793,26888150902,C0019665,854,861,"{'score': -1000, 'preferred': 'Historical aspe..."


In [7]:
test = test.sort_values(['patient_id', 'service_id', 'begin_span'])

In [8]:
# read service data
enc = pd.read_csv(dir_to_parse + 'top-500-encounters-test.txt',sep="|")
enc['encounter_date'] = enc['encounter_date'].astype('datetime64[ns]')
print(enc.head())
print(enc.columns)
enc = enc.sort_values(['patient_id', 'encounter_date', 'service_id'])

   patient_id                                    uuid   service_id  \
0  3822875575  {5020C2C1-655D-44FA-AD2A-E9757F11BFBD}  33900173396   
1  3822875575  {5020C2C1-655D-44FA-AD2A-E9757F11BFBD}  49102456749   
2  3822877342  {DD261B15-4F6C-484A-94FF-9EBCDB697210}  49096193273   
3  3822877342  {DD261B15-4F6C-484A-94FF-9EBCDB697210}  49099288249   
4  3822877999  {4A8831AD-15F1-4C0C-B7EC-19B3808473D3}  42877879066   

  encounter_date  
0     2014-02-26  
1     2015-03-18  
2     2014-12-16  
3     2015-02-05  
4     2014-05-29  
Index(['patient_id', 'uuid', 'service_id', 'encounter_date'], dtype='object')


In [9]:
# import numpy as np

# codes = pd.read_csv(dir_to_parse + 'test_icd9.txt',sep="|")
# print(codes.head(), len(codes))
# print(codes.columns)

# # add random ints for testing
# codes['icd9'] = np.random.randint(0,1000,size=(len(codes),1)) 
# print(codes.head())

#print(codes['icd9'].copy().tolist())

#codes = codes[['patient_id'].isin(patient_id)]

In [10]:
# Operations for random date shift: https://stackoverflow.com/questions/30926323/how-to-do-a-calculation-on-python-with-a-random-operator/30926409
operators = ["+","-"]  
oper = {'+': op.add, '-': op.sub}

In [11]:
def shift_date(df, fn, n):

    df["date_shift"] = fn(df["encounter_date"], timedelta(days=n)).dt.strftime('%Y-%m-%d')
    
    return df

In [12]:
start = time.time()
a = pd.DataFrame()
b = pd.DataFrame()

for i in case_filter:  
    print(i)
    service_ids = test['service_id'][test['patient_id']==i].copy()
   
    # merge encounter dates with notes
    dates = enc.merge(service_ids, left_on="service_id", right_on="service_id", how="inner").drop_duplicates()
    
    # merge icd9 codes with notes
    icd9 = codes.merge(service_ids, left_on="service_id", right_on="service_id", how="inner")
    
    # shift dates by integer selected from randrange
    irand = randrange(1, 365)
    # get random operation for date shift
    fn = oper[random.choice(operators)]
    
    for s in set(service_ids.tolist()):
        
        enc_date = dates[dates['service_id'] == s].copy()
        icdcodes = icd9[icd9['service_id'] == s].copy() 
        
        # deidentified patient_id
        enc_date['uuid'] = enc_date['uuid'].str.replace('}','')
        enc_date['uuid'] = enc_date['uuid'].str.replace('{','')
        
        # shift date by irand and rand operator
        rand_date = shift_date(enc_date, fn, irand)['date_shift']#.values[0]
       
        # merge new date back into original df
        enc_date = enc_date.merge(rand_date.to_frame(), how='inner')
        
        out = test[test['service_id'] == s].copy()
        out.sort_values(by=['begin_span', 'end_span'], inplace=True)
        
        new_artifacts = enc_date.merge(out, how='inner', on='service_id')
        new_icd_code = enc_date.merge(icd9, how='inner', on='service_id')
        
        a = pd.concat([a, new_artifacts[['uuid', 'item', 'date_shift', 'begin_span', 'end_span']].drop_duplicates()])
        b = pd.concat([b, new_icd_code[['uuid', 'dx_code', 'date_shift']].drop_duplicates()])

a.to_csv(dir_to_parse + 'artifacts_500_test.csv', index=False)
b.to_csv(dir_to_parse + 'icd9_500_test.csv', index=False)  

elapsed = (time.time() - start)
print('fini!', 'time:', elapsed)

33213296641
37987612681
3824467978
3823910923
3823626249
3824508942
3822921746
12122093589
3823403033
5149927454
3824154657
5039028258
3823173667
3823753251
21623046181
3823962155
19813328948
3823304759
3822880828
19744069693
3823958082
3823247431
8750948423
3823769675
3823888460
49106223185
5903192147
49106526292
3824132190
3823915103
19744084062
3823155297
3824369764
3823460455
3823214699
3824212077
3823253618
3823958135
3823218808
3824578688
3824113793
3823491221
3823687832
3823859866
3823863962
18825306270
19744082082
3823943842
3824128163
3823282345
8750950569
3822917801
19744077998
3822868656
3824294078
3823288512
3823415493
3824160966
49100564682
3823397069
3822809305
3823847642
3823274206
3824079074
3823147235
3823306985
3824027883
13000192241
3823741172
49107226868
3823024374
13000192255
3823282433
19749165315
3823892742
11978893574
3822917897
3823520010
3823960334
3823347985
49104963859
3823448340
3824115993
19744074020
3823489319
3822895402
16583989550
19786817849
3508294073

In [19]:
len(set(b['uuid'].to_list()))

499

In [ ]:
# import numpy as np

# test = dict()

# df['score'], df['concept'] = df['attributes'].str.split(',', 1).str
# df['concept'] = df['concept'].str[15:-2] # handle extracting element from string
# df = df.drop(['score'], axis=1)
# df['neg'] = np.where(df['negated'] != 'F', '~', '')

# #print(df.head())
# # get "cui:concept" mapping
# # mapping not needed
# #df['CUI'] = [{key: val} for key, val in zip(df.item, df.neg + df.concept)]


# for i in patient_id:  
#     print(i)
#     service_ids = df['service_id'][df['patient_id']==i].copy()
   
#     # merge encounter dates with notes
#     dates = enc.merge(service_ids, left_on="service_id", right_on="service_id", how="inner").drop_duplicates()
    
#     icd9 = codes.merge(service_ids, left_on="service_id", right_on="service_id", how="inner")
    
#     # shift date by randrange: gives you an integral value
#     irand = randrange(1, 365)
#     #picked_operator = random.choice(oper)
#     # get random oepration
#     fn = oper[random.choice(operators)]
    
#     #print(irand, picked_operator)
   
#     # iterate over encounters
#     for s in set(service_ids.tolist()):
        
#         enc_date = dates[dates['service_id'] == s].copy()
#         icdcodes = codes[codes['service_id'] == s].copy()
       
#         # deidentified patient_id
#         enc_date['uuid'] = enc_date['uuid'].str.replace('}','')
#         enc_date['uuid'] = enc_date['uuid'].str.replace('{','')
#         uuid = enc_date['uuid'].values[0]
#         #print('ttest', uuid)
        
#         #print(shift_date(enc_date, picked_operator, irand))
        
#         # shift date by irand and rand operator
#         rand_date = shift_date(enc_date, fn, irand)['date_shift'].values[0]
#         #print('test', rand_date)
        
#         df1 = df[df['service_id'] == s].copy()
#         df1.sort_values(by=['begin', 'end'], inplace=True)
    
#         # get series of CUIS
#         # cuis = df1['CUI'].copy()
#         cuis = df1['item'].copy()
#         icd = icdcodes['icd9'].copy()
        
#         icd = icd.tolist()
#         cuis = cuis.tolist()
       
# #         Not needed
# #
# #         # get count for CUIs 
# #         df2 = df1.groupby('item').first().copy()
# #         df2['n'] = df1['item'].value_counts()
# #         df2.reset_index(inplace=True)
# #         counts = df2[['item','n']].copy()
# #         counts['count'] = [{key: val} for key, val in zip(counts.item, counts.n)]
        
#         # encounter key
#         #s = str(s)+str(rand_date)
#         s = str(uuid)+str(rand_date)
        
        
# #         if str(i) == str(3822760233):
#         if uuid not in test: # if i not in test:
#             # list of dictionaries of cuis, iem counts, respectitvely, to list of single dictionary
#             #a = {k: v for d in cuis.tolist() for k, v in d.items()}
#             #b = {k: v for d in counts['count'].tolist() for k, v in d.items()}
#             a = {'cui': cuis}
#             c = {'icd9': icd}
            
#             #test[i] = {s: [[a, b], rand_date]}
#             #test[uuid] = {s: [c, [a, b], rand_date]}
#             test[uuid] = {s: [c, a, rand_date]}
#         else:
#             #a = {k: v for d in cuis.tolist() for k, v in d.items()}
#             #b = {k: v for d in counts['count'].tolist() for k, v in d.items()}
#             a = {'cui': cuis}
#             c = {'icd9': icd}
#             #test[i].update({s: [[a, b], rand_date]})
#             #test[uuid].update({s: [c, [a, b], rand_date]})
#             test[uuid].update({s: [c, a, rand_date]})
        
        
# # for k, v in dd[3822760232].items():
# #     print(k)
    
# import json
# with open('file_icd_new.txt', 'w') as file:
#      file.write(json.dumps(test))

In [ ]:
#service_ids

#cuis = df[['item', 'attributes']][df['service_id'] == 20322501260].copy()

df['score'], df['concept'] = df['attributes'].str.split(',', 1).str
df1 = df.copy()
df1['concept'] = df1['concept'].str[15:-1].copy()
df1 = df1.drop(['score'], axis=1)
df1['Merged'] = [{key: val} for key, val in zip(df1.item, df1.concept)]
print(df1.head())


In [17]:
len(set(enc['patient_id'].to_list()))

500